### Retina Net

Looking into another pre-trained model, this time we look at a network which tries to balance both accuracy and speed. This model tries to be a middle ground between the Faster RCNN model with ResNet50 and the Faster RCNN model with MobileNet V3 backbones. 

In [1]:
import torchvision.transforms as transforms
from torchvision.models.detection import RetinaNet_ResNet50_FPN_Weights
from torchvision.models.detection import retinanet_resnet50_fpn
from PIL import Image
from os import listdir
from os import path
import numpy as np
from torch import device
from torch import cuda
import cv2
import torch

In [2]:
join = path.join

# image path and annotations path.
img_path = r'/Users/chiahaohsutai/Documents/GitHub/PRW/images/frames'
ann_path = r'/Users/chiahaohsutai/Documents/GitHub/PRW/annotations'

# get the image names.
img_names = sorted(list(listdir(img_path)))
img_names = [join(img_path, name) for name in img_names]

# get the annoation names.
ann_names = sorted(list(listdir(ann_path)))
ann_names = [join(ann_path, name) for name in ann_names]

In [3]:
CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella', 'N/A', 'N/A',
    'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
    'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'N/A', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'N/A', 'dining table',
    'N/A', 'N/A', 'toilet', 'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book',
    'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

# define the torchvision image transforms
transform = transforms.Compose([
    transforms.ToTensor(),
])

def predict(image, model, device, detection_threshold):

    # transform the image to tensor
    image = transform(image).to(device)
    image = image.unsqueeze(0) # add a batch dimension
    outputs = model(image)     # get the predictions on the image

    # get all the predicited class names
    pred_labels = outputs[0]['labels'].cpu().numpy()

    # get score for all the predicted objects
    pred_scores = outputs[0]['scores'].detach().cpu().numpy()

    # get all the predicted bounding boxes
    pred_bboxes = outputs[0]['boxes'].detach().cpu().numpy()

    # get boxes above the threshold score
    boxes = pred_bboxes[pred_scores >= detection_threshold].astype(np.int32)
    labels = pred_labels[pred_scores >= detection_threshold]
    
    # keep only pedestrian predictions.
    boxes = boxes[labels == 1]
    labels = labels[labels == 1]

    return boxes, labels, outputs

In [4]:
def draw_boxes(boxes, labels, image):

    # create a color for the bounding box.
    COLOR = [255, 0, 0] 
  
    # read the image with OpenCV
    image = cv2.cvtColor(np.asarray(image), cv2.COLOR_BGR2RGB)

    # draw only the boxes which are persons.
    for i, box in enumerate(boxes):
        if labels[i] == 1:
            cv2.rectangle(image, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), COLOR, 2)

    return image

In [5]:
# create the model.
weights = RetinaNet_ResNet50_FPN_Weights.DEFAULT
model = retinanet_resnet50_fpn(weights=weights)

In [6]:
# set the device.
device = device('mps' if torch.has_mps else 'cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# make a prediction.
image = Image.open(img_names[0])
model.eval().to(device)
boxes, labels, _ = predict(image, model, device, 0.6)

[W MPSFallback.mm:11] Warning: The operator 'torchvision::nms' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (function operator())


In [8]:
img = draw_boxes(boxes, labels, image)

Results are not as accurate as the Faster RCNN with ResNet50 backbone.

Now we are going to evaluate the models performance using RSME, MAE, UOI.

In [9]:
from collections import namedtuple
import numpy as np
from scipy.io import loadmat
import pandas as pd

In [10]:
# define the `Detection` object
Detection = namedtuple("Detection", ["image_path", "gt", "pred"])

def iou(boxA, boxB):
    """Calculates Intersection Over Union."""
    
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    
    # return the intersection over union value
    return iou

def get_boxes(ann):
    """Gets the correct key for annotations."""
    
    keys = ann.keys()
    key = None
    
    # get the correct key for the bouding box.
    for k in ['box_new', 'anno_file', 'anno_previous']:
        if k in keys:
            key = k
            break
    if key is None:
        raise ValueError("Invalid Annotation Error")
    
    # get the bounding boxes and convert to coordinates.
    bbox = [box[1:] for box in ann[key]]
    for box in bbox:
        xmin, ymin, w, h = box
        xmax, ymax = xmin+w, ymin+h
        box[-2] = xmax
        box[-1] = ymax
        
    return bbox

In [11]:
total_iou = []
# truth, pred
total_count = []
total = len(img_names)

# evaluate the model.
for img, ann in zip(img_names, ann_names):
    
    # get appropiate data.
    data = loadmat(ann)
    photo = Image.open(img)
    gt = get_boxes(data)
    
    # get a prediction
    with torch.no_grad():
        pred = predict(photo, model, device, 0.7)
    
    # go through the prediciton and calulate iou
    boxes, labels, _ = pred
    for bbox in boxes:
        # get the iou.
        union = 0
        for b in gt:
            union = max(union, iou(b, bbox))
        if union > 0.25:
            total_iou.append(union)
    
    # get a count of the boxes.
    total_count.append((len(gt), len(labels)))

In [13]:
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_percentage_error as MAPE

In [14]:
# extract the counts.
y_true = [c[0] for c in total_count]
y_pred = [c[1] for c in total_count]

mse_eval = MSE(y_true, y_pred, squared=False)
mape_eval = MAPE(y_true, y_pred)
avg_iou = sum(total_iou) / len(total_iou)

In [15]:
print(f'RMSE: {mse_eval}, MAPE: {mape_eval}, Avg. IOU: {avg_iou}')

RMSE: 1.7657859439920796, MAPE: 0.3239119047619048, Avg. IOU: 0.8153919267537796
